In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [6]:
import pandas as pd
import numpy as np
data = pd.read_csv("Data_for_UCI_named.csv")
data.head(5)

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
0,2.959060,3.079885,8.381025,9.780754,3.763085,-0.782604,-1.257395,-1.723086,0.650456,0.859578,0.887445,0.958034,0.055347,unstable
1,9.304097,4.902524,3.047541,1.369357,5.067812,-1.940058,-1.872742,-1.255012,0.413441,0.862414,0.562139,0.781760,-0.005957,stable
2,8.971707,8.848428,3.046479,1.214518,3.405158,-1.207456,-1.277210,-0.920492,0.163041,0.766689,0.839444,0.109853,0.003471,unstable
3,0.716415,7.669600,4.486641,2.340563,3.963791,-1.027473,-1.938944,-0.997374,0.446209,0.976744,0.929381,0.362718,0.028871,unstable
4,3.134112,7.608772,4.943759,9.857573,3.525811,-1.125531,-1.845975,-0.554305,0.797110,0.455450,0.656947,0.820923,0.049860,unstable


In [7]:
data.columns

Index(['tau1', 'tau2', 'tau3', 'tau4', 'p1', 'p2', 'p3', 'p4', 'g1', 'g2',
       'g3', 'g4', 'stab', 'stabf'],
      dtype='object')

In [8]:
data.isna().sum()

tau1     0
tau2     0
tau3     0
tau4     0
p1       0
p2       0
p3       0
p4       0
g1       0
g2       0
g3       0
g4       0
stab     0
stabf    0
dtype: int64

In [9]:
data['stabf'] = data['stabf'].replace({'unstable':0,'stable':1})
data.drop('stab',1,inplace=True)

In [11]:
X = data.drop('stabf',1)
y = data['stabf']

In [12]:
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=1)
cols = list(data.columns)[:-1]
scaler = StandardScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.transform(X_test[cols])

0.2636

In [14]:
#Q6 What is the accuracy on the test set using the XGboost classifier? In 4 decimal places.
XGB = XGBClassifier(random_state=1)
XGB.fit(X_train,y_train)
round(XGB.score(X_test,y_test),4)

[12:08:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9455

In [40]:
ETC_1 = ExtraTreesClassifier(random_state=1)
ETC_1.fit(X_train,y_train)
ETC_1.score(X_test,y_test)

0.9285

In [41]:
ETC_2 = ExtraTreesClassifier(random_state=1,n_estimators= 1000,min_samples_split= 2,min_samples_leaf= 8,max_features= None)
ETC_2.fit(X_train,y_train)
ETC_2.score(X_test,y_test)

0.927

In [18]:
#Q9
n_estimators = [1000, 300, 500, 100]
min_samples_split = [2,5,2, 7]
min_samples_leaf = [8,6,8,4 ]
max_features = [None,'auto', 'log2',None] 
hyperparameter_grid = {'n_estimators': n_estimators,
                       'min_samples_leaf': min_samples_leaf,
                       'min_samples_split': min_samples_split,
                       'max_features': max_features}
ETC = ExtraTreesClassifier(random_state=1)
model = RandomizedSearchCV(estimator=ETC,
                          param_distributions=hyperparameter_grid,
                          cv=5,
                           n_iter=10,
                           scoring = 'accuracy', 
                           n_jobs = -1,verbose = 1,
                           random_state = 1)
model.fit(X_train,y_train)
model.best_params_


Fitting 5 folds for each of 10 candidates, totalling 50 fits


{'n_estimators': 1000,
 'min_samples_split': 2,
 'min_samples_leaf': 6,
 'max_features': None}

In [43]:
feature_importance = pd.DataFrame({'Column_nam':cols,'Importance_score':ETC_2.feature_importances_})
feature_importance

,Column_nam,Importance_score
0,tau1,0.137240
1,tau2,0.140508
2,tau3,0.134680
3,tau4,0.135417
4,p1,0.003683
5,p2,0.005337
6,p3,0.005429
7,p4,0.004962
8,g1,0.102562
9,g2,0.107578


In [44]:
feature_importance.sort_values('Importance_score',inplace=True,ascending=False)
feature_importance

,Column_nam,Importance_score
1,tau2,0.140508
0,tau1,0.137240
3,tau4,0.135417
2,tau3,0.134680
10,g3,0.113063
11,g4,0.109541
9,g2,0.107578
8,g1,0.102562
6,p3,0.005429
5,p2,0.005337


# 13


In [32]:
XGB.get_params


<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=4,
              num_parallel_tree=1, predictor='auto', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)>

In [33]:
XGB = XGBClassifier(random_state=1,learning_rate=0.1,max_depth=3)
XGB.fit(X_train,y_train)
round(XGB.score(X_test,y_test),4)

[13:30:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9195

In [29]:
#16 What is the accuracy on the test set using the random forest classifier? In 4 decimal places.
RFC = RandomForestClassifier(random_state=1)
RFC.fit(X_train,y_train)
print('the default scoring metric is accuracy :', round(RFC.score(X_test,y_test),4)) 

the default scoring metric is accuracy : 0.9295


In [30]:
#17 What is the accuracy on the test set using the LGBM classifier? In 4 decimal places.
XGB = XGBClassifier(random_state=1)
XGB.fit(X_train,y_train)
round(XGB.score(X_test,y_test),4)

[13:17:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.9455

In [24]:
#18 
TP = 255
FP = 1380
FN = 45
TN = 20

Precision = TP/(TP+FP)
Recall = TP/(TP+FN)

F1 = 2*((Precision*Recall)/(Precision+Recall))

round(F1,4)


0.2636

In [23]:
#Q20 TN = 99%, FP = 1%, FN = 21%, TP = 79%
recall = 79/(79+21)
fpr = 1/(99+1)
total_price = 5*1+21
recall,fpr,total_price # option 1

(0.79, 0.01, 26)

In [37]:
recall = 78/(78+22)
fpr = 9/(91+9)
total_price = 5*9+22
recall,fpr,total_price # option 2
#TN = 91%, FP = 9%, FN = 22%, TP = 78% 

(0.78, 0.09, 67)

In [35]:
recall = 90/(90+10)
fpr = 4/(96+4)
total_price = 5*4+10
recall,fpr,total_price # option3

#TN = 96%, FP = 4%, FN = 10%, TP = 90% 

(0.9, 0.04, 30)

In [34]:
recall = 82/(82+18)
fpr = 2/(98+2)
total_price = 5*2+18
recall,fpr,total_price # option 4
#TN = 98%, FP = 2%, FN = 18%, TP = 82%

(0.82, 0.02, 28)